<a href="https://colab.research.google.com/github/Fikri2272/Jawaban-Soal-Nomor-3-5/blob/main/Jawaban_Nomor_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving heart_modified.csv to heart_modified.csv


In [15]:
# ====== NOMOR 3: PRAPROSES DATA ======

# 3.a Pisahkan atribut prediktor dan label
label_col = "HeartDisease"
X = df.drop(columns=[label_col])
y = df[label_col]

# Bagi ke numeric dan kategorik
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Fitur numerik:", numeric_cols)
print("Fitur kategorik:", categorical_cols)

# ---------- 3.b SimpleImputer (mean) untuk numerik ----------
imputer = SimpleImputer(strategy="mean")
X_num_imputed = pd.DataFrame(imputer.fit_transform(X[numeric_cols]),
                             columns=numeric_cols)

print("\nMissing value setelah imputasi (numerik):")
print(X_num_imputed.isna().sum())

# ---------- 3.c StandardScaler untuk numerik ----------
scaler = StandardScaler()
X_num_scaled = pd.DataFrame(scaler.fit_transform(X_num_imputed),
                            columns=numeric_cols)

print("\nStatistik deskriptif numerik SETELAH scaling:")
print(X_num_scaled.describe())

# ---------- 3.d Praproses tambahan: One-Hot Encoding kategorik ----------
X_cat_encoded = pd.get_dummies(X[categorical_cols], drop_first=False)

print("\nContoh hasil one-hot encoding (5 baris pertama):")
print(X_cat_encoded.head())

# ---------- Gabungkan numerik ter-scale + kategorik ter-encode ----------
X_processed = pd.concat([X_num_scaled, X_cat_encoded], axis=1)

print("\nShape after preprocessing:", X_processed.shape)
print("Statistik deskriptif numerik SEBELUM praproses:")
print(df[numeric_cols].describe())

Fitur numerik: ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
Fitur kategorik: ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

Missing value setelah imputasi (numerik):
Age            0
RestingBP      0
Cholesterol    0
FastingBS      0
MaxHR          0
Oldpeak        0
dtype: int64

Statistik deskriptif numerik SETELAH scaling:
                Age     RestingBP   Cholesterol     FastingBS         MaxHR  \
count  9.180000e+02  9.180000e+02  9.180000e+02  9.180000e+02  9.180000e+02   
mean  -1.083616e-16  7.275710e-16 -3.096047e-17 -3.096047e-17 -7.894919e-16   
std    1.000545e+00  1.000545e+00  1.000545e+00  1.000545e+00  1.000545e+00   
min   -2.706015e+00 -7.189253e+00 -1.836365e+00 -5.513413e-01 -3.039576e+00   
25%   -6.906294e-01 -6.738605e-01 -2.189072e-01 -5.513413e-01 -6.672068e-01   
50%    5.188098e-02 -1.309111e-01  2.096499e-01 -5.513413e-01  2.473414e-02   
75%    6.883185e-01  4.120382e-01  6.220785e-01 -5.513413e-01  7.166751e

In [12]:
# ====== NOMOR 4: MODEL KLASIFIKASI (LOGISTIC REGRESSION) ======

# Import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# === 1. Baca dataset ===
df = pd.read_csv("heart_modified.csv")

# Tentukan kolom label
label_col = "FastingBS"      # ubah sesuai label di datasetmu
X = df.drop(columns=[label_col])
y = df[label_col]

# === 2. Pisahkan kolom numerik dan kategorik ===
numeric_cols = X.select_dtypes(include=[np.number]).columns
categorical_cols = X.select_dtypes(exclude=[np.number]).columns

print("Kolom numerik :", numeric_cols.tolist())
print("Kolom kategorik :", categorical_cols.tolist())

# === 3. Buat pipeline praproses ===
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

# === 4. Bagi data (85% train, 15% test) ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42,
    stratify=y
)

# === 5. Pipeline akhir: preprocessing + model ===
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# === 6. Latih model ===
clf.fit(X_train, y_train)

# === 7. Prediksi ===
y_pred = clf.predict(X_test)

# === 8. Evaluasi model ===
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("\nConfusion Matrix:\n", cm)
print("\nAkurasi Model :", round(acc, 4))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))



Kolom numerik : ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak', 'HeartDisease']
Kolom kategorik : ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

Confusion Matrix:
 [[102   4]
 [ 25   7]]

Akurasi Model : 0.7899

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.80      0.96      0.88       106
           1       0.64      0.22      0.33        32

    accuracy                           0.79       138
   macro avg       0.72      0.59      0.60       138
weighted avg       0.76      0.79      0.75       138



In [14]:
# ====== NOMOR 5: MODEL CLUSTERING (K-MEANS) ======

# Import library yang dibutuhkan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Pastikan variabel X_processed sudah ada dari tahap praproses sebelumnya.
# Jika belum, jalankan kembali bagian praproses (OneHotEncoder + StandardScaler)
# atau gunakan pipeline yang sama seperti di nomor 4.

# Kalau kamu sudah punya pipeline preprocessor dari nomor 4:
X_cluster_ready = preprocessor.fit_transform(X)   # hasil numeric, siap untuk clustering

# 5.a Bangun model clustering K-Means
# Kita pilih k = 2 karena kasus HeartDisease biasanya dua kelompok (berisiko vs tidak)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)

cluster_labels = kmeans.fit_predict(X_cluster_ready)

# 5.b Hitung SSE dan silhouette score
sse = kmeans.inertia_                 # Sum of Squared Errors (total jarak kuadrat ke pusat cluster)
sil = silhouette_score(X_cluster_ready, cluster_labels)

print("SSE (Sum of Squared Errors):", round(sse, 4))
print("Silhouette Score:", round(sil, 4))


SSE (Sum of Squared Errors): 6001.762
Silhouette Score: 0.2227
